In [2]:
import pandas as pd
import openpyxl
import ht
import numpy as np


def load_data_table():
    url ='data_tables.csv'

    return pd.read_csv(url)
thickness_table = load_data_table().iloc[11:36,1:4]
thickness_table.columns = thickness_table.iloc[0]
thickness_table = thickness_table[1:] #.set_index('Gauge')
print(thickness_table)
print(thickness_table.columns.names)

workbook = openpyxl.load_workbook('datasheet_bell_tiral.xlsx', data_only=True)
worksheet = workbook['Sheet1']

s3 = worksheet['I10'].value
t1_s = worksheet['H20'].value
t2_s =worksheet['I20'].value
m_s = worksheet['H14'].value
Cp_s = worksheet['H24'].value
mu_s = worksheet['H22'].value*0.001
rho_s =  worksheet['H21'].value
k_s = worksheet['H25'].value
fouling_s = worksheet['H30'].value
mu_t = worksheet['K22'].value*0.001
fouling_t =  worksheet['K30'].value
rho_t =  worksheet['K21'].value
m_t = worksheet['K14'].value
t1_t =  worksheet['K20'].value
t2_t = worksheet['L20'].value
Cp_t = worksheet['K24'].value
k_t =worksheet['K25'].value

shell_side ='Cold Side'
s2 = 'Cold side T2'
if shell_side == 'Hot Side':
    T1 = t1_s  
    T2 =t2_s
    m_c = m_t
    
    m_h = m_s
    t1 = t1_t
    t2 = t2_t
    Cp_h = Cp_s
    Cp_c = Cp_t
    
else:
    T1 = t1_t  
    T2 =t2_t
    m_c = m_s
    m_h = m_t
    t1 = t1_s
    t2 = t2_s
    Cp_h = Cp_t
    Cp_c = Cp_s
if s2 == 'Hot side mass flow':
    Q = m_c * Cp_c * (t2-t1)
    m_h = Q/((Cp_h*(T1-T2)))
elif s2 == 'Hot side T1':
    Q = m_c * Cp_c * (t2-t1)
    T1 = T2 + (Q/(m_h*Cp_h))
elif s2 == 'Hot side T2':
    Q = m_c * Cp_c * (t2-t1)
    T2 = T1 - (Q/(m_h*Cp_h))
elif s2 == 'Cold side mass flow':
    Q = m_h * Cp_h * (T1-T2)
    m_c = Q/((Cp_c*(t2-t1))) 
elif s2 == 'Cold side T1':
    Q = m_h * Cp_h * (T1-T2)
    t1 = t2 - (Q/(m_c*Cp_c))
else: #cold side T2
    Q = m_h * Cp_h * (T1-T2)
    t2 = t1 + (Q/(m_c*Cp_c))
dTlm = ht.LMTD(T1,T2,t1,t2)
pn = worksheet['H37'].value
ft = ht.F_LMTD_Fakheri(t1,t2,T1,T2,1)
Do = worksheet['F42'].value
thick = float(thickness_table[thickness_table['Gauge']==str(worksheet['H42'].value)]['mm']) #2.108
print(worksheet['H42'].value)
print(thick)
Di = (Do - 2*thick)*0.001

shell_D = worksheet['H44'].value/1000
tn = worksheet['D42'].value

#Do = float(geo_df.iloc[2,1])
#Di = (Do - 2*float(geo_df.iloc[3,1]))*0.001
L = worksheet['J42'].value
tpitch = Do*worksheet['M43'].value
b_space = worksheet['M48'].value
b_cut = worksheet['I48'].value
print(tn,shell_D,b_cut,b_space,tpitch,b_cut)


pitch ='square'
if pitch == 'square':
    De = 4*(((tpitch*0.001)**2)-(3.14*((Do*0.001)**2)*0.25))/(3.14*Do*0.001)
else:
    De = 8*(0.43301*((tpitch*0.001)**2)-(3.14*((Do*0.001)**2)*0.125))/(3.14*Do*0.001)
C = tpitch-Do
As = (shell_D*b_space*C)/(tpitch*1000)
Gs = m_h/(As*3600)
Res = (De*Gs)/mu_s
f = np.exp(0.576-(0.19*np.log(Res)))
Nb = (L/b_space)-1
dp_s = ((f*(Gs**2)*(Nb+1)*shell_D)/(2*rho_s*De))*0.000010197
L = L/1000
A = np.pi*L*Do*0.001*1*tn
Cp_t = Cp_t*4184
Cp_s = Cp_s*4184
cross_A=(np.pi*0.25*(Di**2))*(tn/pn)
velocity_t = m_c/(rho_t*3600*cross_A)
Ret=(rho_t*velocity_t*Di)/mu_t
f_t =1/(1.58*np.log(Ret)-3.28)**2 # valid for Re 2300 to 5,000,000 and Pr 0.5 to 2000
port_1 = f_t*L*pn/Di
port_2 = rho_t*(velocity_t**2)/2
dp_t = (4*(port_1)+4*(pn))*port_2*0.000010197
h_shell = (0.36*((Res)**0.55)*((Cp_s*mu_s/k_s)**(1/3)))*k_s/De #for Re between 2000 and 1,000,000
Pr = Cp_t*mu_t/k_t
Nu = ((0.5*f_t*(Ret-1000)*Pr))/(1+12.7*((0.5*f_t)**0.5)*((Pr**(2/3))-1)) # valid for Re 2300 to 5,000,000 (Gnielinski)
h_t = Nu *k_t/Di
d_ratio = Do/(Di*1000)
Uc = 1/((d_ratio/h_t)+(Do*0.001*np.log(d_ratio)/(2*60))+(1/h_shell))
Ud = 1/((d_ratio/h_t)+(Do*0.001*np.log(d_ratio)/(2*60))+(1/h_shell)+fouling_s+(d_ratio*fouling_t))
U_calc = Q/(ft*dTlm*A)
Rdesign = - (1/Uc) + (1/Ud)
Rsevice = - (1/Uc) + (1/U_calc)
j_const = pd.read_csv('j_consts.csv')
a1 = np.where((Res < j_const['Reynolds_max']) & (Res < j_const['Reynolds_max']) & (j_const['Layout'] == 30) ,j_const['a_{1}'],0).sum()
a2 = np.where((Res < j_const['Reynolds_max']) & (Res < j_const['Reynolds_max']) & (j_const['Layout'] == 30) ,j_const['a_{2}'],0).sum()
a3 = np.where((Res < j_const['Reynolds_max']) & (Res < j_const['Reynolds_max']) & (j_const['Layout'] == 30) ,j_const['a_{3}'],0).sum()
a4 = np.where((Res < j_const['Reynolds_max']) & (Res < j_const['Reynolds_max']) & (j_const['Layout'] == 30) ,j_const['a_{4}'],0).sum()
b1 = np.where((Res < j_const['Reynolds_max']) & (Res < j_const['Reynolds_max']) & (j_const['Layout'] == 30) ,j_const['b_{1}'],0).sum()
b2 = np.where((Res < j_const['Reynolds_max']) & (Res < j_const['Reynolds_max']) & (j_const['Layout'] == 30) ,j_const['b_{2}'],0).sum()
b3 = np.where((Res < j_const['Reynolds_max']) & (Res < j_const['Reynolds_max']) & (j_const['Layout'] == 30) ,j_const['b_{3}'],0).sum()
b4 = np.where((Res < j_const['Reynolds_max']) & (Res < j_const['Reynolds_max']) & (j_const['Layout'] == 30) ,j_const['b_{4}'],0).sum()
a = a3/(1+0.14*(Res**a4))
b = b3/(1+0.14*(Res**b4))
j = a1 * ((1.33/(tpitch/Do))**a)*(Res**a2)
h_ideal = j*Cp_s*(m_s/(3600*As))*(k_s/(Cp_s*mu_s))**(2/3)
print(h_ideal)


11 Gauge    BWG    mm
12     4  0.238  6.05
13     5   0.22  5.59
14     6  0.203  5.16
15     7   0.18  4.57
16     8  0.165  4.19
17     9  0.148  3.76
18    10  0.134   3.4
19    11   0.12  3.05
20    12  0.109  2.77
21    13  0.095  2.41
22    14  0.083  2.11
23    15  0.072  1.83
24    16  0.065  1.65
25    17  0.058  1.47
26    18  0.049  1.24
27    19  0.042  1.07
28    20  0.035  0.89
29    21  0.032  0.81
30    22  0.028  0.71
31    23  0.025  0.64
32    24  0.022  0.56
33    25   0.02  0.51
34    26  0.018  0.46
35    27  0.016  0.41
[11]
14
2.11
414 0.609 25 234 23.8125 25
3378.6711967118417


In [3]:
import pandas as pd
import numpy as np 
import openpyxl
import ht
j_const = pd.read_csv('j_consts.csv')
calc_list = ['Duty','LMTD','Ft','Corrected LMTD','Surface Area','Tube Heat transfer Coef.','Shell Heat transfer Coef.','Uclean','Udirty','Uservice','Over Design','Over Surface','Shell Pressure Drop','Tube Pressure Drop','Shell Reynolds Number','Tube Reynolds Number','Tube Velocity','Shell Velocity']
geo_input_list = ['Shell D','Baffle Spacing','Number of baffles','Do','Di','Length','Number of tubes','Number of passes','Tube pitch','pitch type']
para_input_list = ['Tube Flow rate','Tube inlet temperature','Tube outlet temperature','Tube Density','Tube Heat Capacity','Tube Viscosity', 'Tube Thermal conductivity','Tube Fouling factor','Shell flow rate','Shell inlet temperature','Shell outlet temperature','Shell Density','Shell Heat Capacity','Shell Viscosity', 'Shell Thermal conductivity','Shell Fouling factor'] 
calculations_df = pd.DataFrame(index=calc_list)
geo_input_df = pd.DataFrame(index=geo_input_list)
para_input_df = pd.DataFrame(index=para_input_list)   
def Heat_balance(shell_side, Tube_list, Shell_list,s2,s3):
    m_t,t1_t,t2_t,rho_t,Cp_t,mu_t,k_t,fouling_t = Tube_list[0], Tube_list[1], Tube_list[2], Tube_list[3], Tube_list[4], Tube_list[5], Tube_list[6], Tube_list[7]
    m_s,t1_s,t2_s,rho_s,Cp_s,mu_s,k_s,fouling_s = Shell_list[0], Shell_list[1], Shell_list[2], Shell_list[3], Shell_list[4], Shell_list[5], Shell_list[6], Shell_list[7]
    if shell_side == 'Hot Side':
        T1 = t1_s  
        T2 =t2_s
        m_c = m_t
        m_h = m_s
        t1 = t1_t
        t2 = t2_t
        Cp_h = Cp_s
        Cp_c = Cp_t
    else:
        T1 = t1_t  
        T2 =t2_t
        m_c = m_s
        m_h = m_t
        t1 = t1_s
        t2 = t2_s
        Cp_h = Cp_t
        Cp_c = Cp_s
    if s2 == 'Hot side mass flow':
        Q = m_c * Cp_c * (t2-t1)
        m_h = Q/(Cp_h*(T1-T2))
    elif s2 == 'Hot side T1':
        Q = m_c * Cp_c * (t2-t1)
        T1 = T2 + (Q/m_h*Cp_h)
    elif s2 == 'Hot side T2':
        Q = m_c * Cp_c * (t2-t1)
        T2 = T1 - (Q/m_h*Cp_h)
    elif s2 == 'Cold side mass flow':
        Q = m_h * Cp_h * (T1-T2)
        m_c = Q/(Cp_c*(t2-t1)) 
    elif s2 == 'Cold side T1':
        Q = m_h * Cp_h * (T1-T2)
        t1 = t2 - (Q/m_c*Cp_c)
    else: #cold side T2
        Q = m_h * Cp_h * (T1-T2)
        t2 = t1 + (Q/m_c*Cp_c)
    dTlm = ht.LMTD(T1,T2,t1,t2)
    Q = Q *1.163 # Kcal to W
    ft = ht.F_LMTD_Fakheri(t1,t2,T1,T2,s3)
    HB_data = [Q,dTlm,ft]  
    return HB_data
def kern(Tube_list, Shell_list, geo_list,s3,HB_data,geo_input_df,calculations_df):
            m_t,t1_t,t2_t,rho_t,Cp_t,mu_t,k_t,fouling_t = Tube_list[0], Tube_list[1], Tube_list[2], Tube_list[3], Tube_list[4], Tube_list[5], Tube_list[6], Tube_list[7]
            m_s,t1_s,t2_s,rho_s,Cp_s,mu_s,k_s,fouling_s = Shell_list[0], Shell_list[1], Shell_list[2], Shell_list[3], Shell_list[4], Shell_list[5], Shell_list[6], Shell_list[7]
            Di,Do,tn,pn,L,tpitch,pitch,b_cut,shell_D,b_space = geo_list[3], geo_list[2], geo_list[0], geo_list[1], geo_list[6], geo_list[5], geo_list[4], geo_list[8], geo_list[-1], geo_list[7]
            Q, dTlm, ft = HB_data[0], HB_data[1], HB_data[2]
            if pitch == 'square' or 'rotated square 45':
              De = 4*(((tpitch*0.001)**2)-(3.14*((Do*0.001)**2)*0.25))/(3.14*Do*0.001)
            else:
              De = 8*(0.43301*((tpitch*0.001)**2)-(3.14*((Do*0.001)**2)*0.125))/(3.14*Do*0.001)
            
            C = tpitch-Do
            As = (shell_D*b_space*C)/(tpitch*1000)
            Gs = m_s/(As*3600)
            velocity_s = Gs/rho_s
            Res = (De*Gs)/mu_s
            f = np.exp(0.576-(0.19*np.log(Res)))
            Nb = (L/b_space)-1
            dp_s = ((f*(Gs**2)*(Nb+1)*shell_D)/(2*rho_s*De))*0.000010197
            L = L/1000
            A = np.pi*L*Do*0.001*s3*tn
            Cp_t = Cp_t*4184
            Cp_s = Cp_s*4184
            cross_A=(np.pi*0.25*(Di**2))*(tn/pn)
            velocity_t = m_t/(rho_t*3600*cross_A)
            Ret=(rho_t*velocity_t*Di)/mu_t
            f_t =1/(1.58*np.log(Ret)-3.28)**2 # valid for Re 2300 to 5,000,000 and Pr 0.5 to 2000
            port_1 = f_t*L*pn/Di
            port_2 = rho_t*(velocity_t**2)/2
            dp_t = (4*(port_1)+4*(pn))*port_2*0.000010197
            h_shell = (0.36*((De*Gs/mu_s)**0.55)*((Cp_s*mu_s/k_s)**(1/3)))*k_s/De #for Re between 2000 and 1,000,000
            Pr = Cp_t*mu_t/k_t
            Nu = ((0.5*f_t*(Ret-1000)*Pr))/(1+12.7*((0.5*f_t)**0.5)*((Pr**(2/3))-1)) # valid for Re 2300 to 5,000,000 (Gnielinski)
            h_t = Nu *k_t/Di
            d_ratio = Do/(Di*1000)
            Uc = 1/((d_ratio/h_t)+(Do*0.001*np.log(d_ratio)/(2*60))+(1/h_shell))
            Ud = 1/((d_ratio/h_t)+(Do*0.001*np.log(d_ratio)/(2*60))+(1/h_shell)+fouling_s+(d_ratio*fouling_t))
            U_calc = Q/(ft*dTlm*A)
            Rdesign = - (1/Uc) + (1/Ud)
            Rsevice = - (1/Uc) + (1/U_calc)
            OD_k=100*((Ud-U_calc)/Ud)
            OV = ((Uc/U_calc)-1)*100
            geo_input_df.loc['Number of baffles','Kern_summary'] = Nb
            calculations_df.loc[['Surface Area','Tube Heat transfer Coef.','Shell Heat transfer Coef.','Uclean','Udirty','Uservice','Shell Pressure Drop','Tube Pressure Drop','Shell Reynolds Number','Tube Reynolds Number','Tube Velocity','Shell Velocity','Over Design','Over Surface'],'Kern_summary']= A,h_t,h_shell,Uc,Ud,U_calc,dp_s,dp_t,Res,Ret,velocity_t,velocity_s,OD_k, OV
            return dp_s, dp_t, h_shell, h_t, Uc, Ud, U_calc, Rdesign, Rsevice
def bell_delaware(Tube_list, Shell_list ,h_t,h_shell,geo_list,s3,HB_data, geo_input_df, calculations_df):
    no_of_shells = s3
    m_t,t1_t,t2_t,rho_t,Cp_t,mu_t,k_t,fouling_t = Tube_list[0], Tube_list[1], Tube_list[2], Tube_list[3], Tube_list[4], Tube_list[5], Tube_list[6], Tube_list[7]
    m_s,t1_s,t2_s,rho_s,Cp_s,mu_s,k_s,fouling_s = Shell_list[0], Shell_list[1], Shell_list[2], Shell_list[3], Shell_list[4], Shell_list[5], Shell_list[6], Shell_list[7]
    Di,Do,tn,pn,L,tpitch,pitch,b_cut,shell_D,b_space = geo_list[3], geo_list[2], geo_list[0], geo_list[1], geo_list[6]/1000, geo_list[5], geo_list[4], geo_list[8], geo_list[-1], geo_list[7]
    # Tube pitch layout
    if pitch == 'square':
      t_p_angle = 45
    elif pitch == 'rotated square 45':
      t_p_angle = 90
    else:
      t_p_angle = 30
    p_ratio = [1.25,1.285,1.33,1.5]
    N_ss = 2 # number_of_sealing_strips 
    T_wall = (t1_t+t2_t)*0.5+h_shell*((t2_t+t2_s)*0.5-(t1_t+t2_t)*0.5)/(h_shell+h_t)
    Di = Di*1000
    print('dp for T_wall '+str(T_wall))
    mu_s_w = 0.59
    mu_t_w = 0.16
    shell_D = shell_D*1000
  
    k_s = k_s/1.16
    k_t = k_t /1.16
    fouling_s = fouling_s*1.16
    fouling_t = fouling_t*1.16
    #Shell outer tube limit
    D_otl = shell_D - (12.5+(shell_D/200))
    #Height of baffle cut
    L_c = b_cut/(100*shell_D)
    # Diametral Sheel baffle clearance
    D_sb = 3.1+0.004*shell_D
    # Tube sheet thickness
    L_s = 0.1*shell_D
    Lb_cut = b_space # central baffle spacing
    LB_in = b_space # inlet baffle spacing
    LB_out = b_space # Outlet baffle spacing
    L_tb = 0.4 # Diametral Tube-Baffle Clearance
    
    # tube arrangement
    if t_p_angle == 30:
      t_arrg = np.sqrt(3)/2
    elif t_p_angle == 45:
      t_arrg = 1/np.sqrt(2)
    else : t_arrg = 1


    #LMTD and Q
    LMTD = HB_data[1] 
    f_t = HB_data[2]
    corrected_LMTD = LMTD * f_t
    Q = HB_data[0]

    #Tube wall temperature
    T_wall = (t1_t+t2_t)*0.5+h_shell*((t1_s+t2_s)*0.5-(t1_t+t2_t)*0.5)/(h_shell+h_t)


    # Shell Side heat transfer coefficient
    t_p = tpitch #p_ratio *Do
    #t_p_effective 
    if t_p_angle == 45:
      t_p_effective = t_p/(2**0.5)
    else: t_p_effective = t_p

    #cross flow area 
    S_m = (Lb_cut/1000)*((shell_D-D_otl)+(D_otl-Do)*(t_p-Do)/t_p_effective)/1000
    Gs = (m_s/3600)/S_m
    Re_s = (Do/1000)*Gs/(mu_s*0.001)
    print('dp for Re_s '+str(Re_s))
    Pr_s = (mu_s/1000)*Cp_s/k_s*3600
    # j- ideal factor coefficients
    a1 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['a_{1}'],0).sum()
    a2 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['a_{2}'],0).sum()
    a3 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['a_{3}'],0).sum()
    a4 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['a_{4}'],0).sum()
    a=a3/(1+0.14*((Re_s)**a4))
    #print(a)
    #print(Re_s)
    j=a1*((1.33/(t_p/Do))**a)*(Re_s**a2)
    print('dp for j '+str(j))
    #print(j)
    h_s_i = Cp_s*(m_s/S_m)*j*(Pr_s**(-2/3))*((mu_s/mu_s_w)**0.14)
    print(h_s_i)
    # 1. correction factor for baffle window flow

    theta_CTL = 2*np.arccos(shell_D*(1-2*b_cut/100)/(D_otl-Do)) #b_cut baffle cut, shell_D isnide shell diameter mm
    F_w = (theta_CTL-np.sin(theta_CTL))/(2*np.pi)

    F_c = 1-2*F_w # Fraction of tubes in cross flow
    j_c = 0.55+0.72*F_c

    # 2. correction factor for baffle leakage
    theta_Ds = 2*np.arccos(1-(2*b_cut)/100) # Baffle window angle
    S_sb = (shell_D/1000)*(D_sb/1000)*(np.pi-0.5*theta_Ds)    # Shell to baffle leakage area
    S_tb = (np.pi/4)*((((Do/1000)+(L_tb/1000))**2)-((Do/1000)**2))*tn*(1-F_w)  # Tube to baffle leakage area

    r_L = (S_sb+S_tb)/S_m # ratio of leakage to cross flow
    r_S = S_sb/(S_sb+S_tb) # ratio of shell-baffle to total area

    j_l = 0.44*(1-r_S)+(1-0.44*(1-r_S))*np.exp(-2.2*r_L) # Baffle leakage correction factor

    # 3. correction factor for bundle bypass

    P_p = t_p * t_arrg # Tube row distance in flow direction
    N_TCC = (shell_D/P_p)*(1-(2*b_cut)/100) #N_TCC number of tube rows between baffle
    r_ss = N_ss/N_TCC # Nss Number of sealing strips
    S_b = (Lb_cut/1000)*(shell_D-D_otl-(Do/2))/1000 #bundle pybass area
    if Re_s < 100:
      C_j = 1.35
    else: C_j = 1.25 # Correlation constant

    if r_ss >= 0.5:
      j_b = 1
    else: j_b = np.exp(-1*C_j*(S_b/S_m)*(1-((2*r_ss)**(1/3))))

    # 4. Correction factor for adverse temperature gradient
    N_tcw = (0.8/P_p)*(shell_D*b_cut/100-(shell_D-(D_otl-Do))/2)
    N_b = 1 +int((L-(2*L_s*0.001)-(LB_in+LB_out)*0.001)/(Lb_cut*0.001)) # number of baffles
    print('value for N_b '+str(N_b))
    N_c = (N_tcw +N_TCC)*(1+N_b) # tube rows crossed in entire exchanger
    j_RL = (10/N_c)**0.18
    if Re_s <= 20:
      j_R = j_RL
    elif Re_s <100:
      j_R = j_RL+((20-Re_s)/80)*(j_RL-1)
    else: j_R = 1

    # 5. correction factor for unequal baffle spacing
    if Re_s < 100:
      n1 = 1/3
    else: n1 = 0.6
    j_s =((N_b-1)+(LB_in/Lb_cut)**(1-n1)+(LB_out/Lb_cut)**(1-n1))/((N_b-1)+(LB_in/Lb_cut)+(LB_out/Lb_cut))


    h_shell = j_s * j_R *j_b * j_l * j_c * h_s_i
    print(h_shell)

    ### Tube Side Heat transfer coeficient
    a_tube = (np.pi*((Di/1000)**2)*tn)/(4*pn) # Flow area
    v_t = (m_t/3600)/(rho_t*a_tube) #velocity through tube
    print('value for v_t '+str(v_t))
    Re_t = (Di/1000)*rho_t*v_t/(mu_t/1000)
    print('value for Re_t '+str(Re_t))
    Pr_t = Cp_t*(mu_t/1000)/k_t*3600
    print('value for Pr_t '+str(Pr_t))
    L_eff = L-2*L_s/1000 # Effective tube length

    # Nusselt Number Calculation
    Nu_laminar = 1.86*(Re_t*Pr_t*(Di/1000)/L_eff)**(1/3)
    # Turbulent flow Petukhov-Kirillov
    f_turbulent = (1.58*np.log(Re_t)-3.28)**-2
    print('value for f_turbulent '+str(f_turbulent))
    Nu_turb = (f_turbulent/2)*Re_t*Pr_t/(1.07+12.7*((f_turbulent/2)**0.5)*((Pr_t**(2/3))-1))
    # Transition flow Nu
    Nu_2300 = 1.86*(2300*Pr_t*(Di/1000)/L_eff)**(1/3)

    f_Re_10000 = (1.58*np.log(10000)-3.28)**-2
    Nu_10000 = (f_Re_10000/2)*10000*Pr_t/(1.07+12.7*((f_Re_10000/2)**0.5)*((Pr_t**(2/3))-1))

    Nu_trans = Nu_2300+(Nu_10000-Nu_2300)*(Re_t-2300)/(10000-2300)


    if Re_t <= 2300:
      Nu_tube = Nu_laminar
    elif Re_t < 10000:
      Nu_tube = Nu_trans
    else: Nu_tube = Nu_turb
    print('value for Nu_tube '+str(Nu_tube))
    h_t_i=Nu_tube*k_t/(Di/1000)*(mu_t/mu_t_w)**0.14
    print('value for h_t_i '+str(h_t_i))

    # Overall Heat tansfer coefficient
    dict_of_conductivity = {'Carbon Steel':38.69,'Copper':324.42,'Inconel':12.95,'Monel':21.28,'Nickel':52.09,'Stainless Steel':13.54}
    k_w_t = dict_of_conductivity['Carbon Steel']
    T_wall = (t1_t+t2_t)/2+h_s_i*((t1_s+t2_s)/2-(t1_t+t2_t)/2)/(h_s_i+h_t_i)
    wall_resistance = (Do/2000)*np.log(Do/Di)/k_w_t


    U_clean = 1/((1/h_shell)+(Do/(h_t_i*Di))+wall_resistance)
    print('dp for U_clean '+str(U_clean))
    U_dirty = 1/((1/U_clean)+fouling_t+fouling_s)
    print('dp for U_dirty '+str(U_dirty))
    ### Shell side pressure drop
    b1 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{1}'],0).sum()
    b2 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{2}'],0).sum()
    b3 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{3}'],0).sum()
    b4 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{4}'],0).sum()
    b = b3/(1+0.14*(Re_s**b4))
    f_s = b1*((1.33/(t_p/Do))**b)*(Re_s**b2)
    dp_shell_ideal = 2*f_s*(Gs**2)*N_TCC*(mu_s_w/mu_s)**0.14/(rho_s)/100000 #N_TCC number of tube rows between baffle
    #print(dp_shell_ideal)

    # 1. correction factor for baffle leakage
    f_p = 0.8-0.15*(1+r_S) # r_s ratio of shell-baffle to total area
    R_L = np.exp(-1.33*(1+r_S)*(r_L**f_p))# r_l ratio of  leakage area to cross flow

    # pressure drop for an ideal window section
    S_wg = (((shell_D/1000)**2)/8)*(theta_Ds-np.sin(theta_Ds)) #Gross window area
    S_wt = tn*F_w*np.pi*((Do/1000)**2)/4 #Window area occupied with tubes
    S_w = S_wg - S_wt # Net Cross flow area
    G_w = (m_s/3600)/np.sqrt(S_m*S_w)
    v_w = G_w/rho_s
    print('dp for G_w '+str(G_w))
    D_w = 4*S_w/(np.pi*(Do/1000)*tn*F_w+(shell_D/1000)*theta_Ds)

    #pressure drop for turbluent flow in ideal window section
    if Re_s >= 100:
      dp_window = N_b*R_L*(2+0.6*N_tcw)*(G_w**2)/(2*rho_s)/100000

      
    else:
      dp_window = N_b*R_L*((26*G_w*Cp_s/rho_s)*(N_tcw/(tpitch-Do)/1000 + (Lb_cut/1000)/(D_w**2)) + (G_w**2)/rho_s)/100000
        

    # 2. Correction factor for bundle bypass effect
    if Re_s < 100:
      C_r = 4.5
    else: C_r = 3.7

    if r_ss >= 0.5:
      R_b = 1
    else:
      R_b = np.exp(-1*C_r*(S_b/S_m)*(1-(2*r_ss)**(1/3)))

    # 3. Correction for unequal baffle spacing inlet/ outlet
    if Re_s < 100:
      n = 1 # Slope of friction factor curve
    else: n = 0.2
    R_s = (Lb_cut/LB_in)**(2-n) + (Lb_cut/LB_out)**(2-n) # correction factor

    # Shell side pressure drop (Excluding nozzles)
    dp_cent_baff = dp_shell_ideal*(N_b-1)*R_L*R_b # pressure drop in baffle windows
    dp_baff_window = dp_window # pressure drop in baffle windows
    dp_entrance_exit = dp_shell_ideal*R_s*R_b*(1+N_tcw/N_TCC) # pressure drop in entrance / exit baffles
    print('dp for cent baffle '+str(dp_cent_baff))
    print('dp for baffle wind '+str(dp_baff_window))
    print('dp for entrance '+str(dp_entrance_exit))

    total_dp_shell = dp_cent_baff + dp_baff_window + dp_entrance_exit
    print('overall shell dp is {}'.format(total_dp_shell))
    # Tube side pressure drop
    total_dp_tube =((4*f_turbulent*L*pn/(Di/1000))+4*pn)*rho_t*(v_t**2)/2/100000
    print('overall Di is {}'.format(Di))
    ### Area Calculations
    print('dp for corrected_LMTD '+str(corrected_LMTD))
    print('dp for LMTD '+str(LMTD))
    print(corrected_LMTD/LMTD)
    A_required = Q/(corrected_LMTD*U_dirty*1.163)
    A_available = L_eff * tn *np.pi*(Do/1000)
    U_required = Q/(corrected_LMTD*A_available*1.163)
    A_over = ((U_clean/U_required)-1)*100
    OD = ((U_dirty/U_required)-1)*100
    print('dp for A_required '+str(A_required))
    print('dp for A_available '+str(A_available))
    print('dp for U_required '+str(U_required))
    print('od is {} while OV is {}'.format(OD,A_over))
    geo_input_df.loc[['Number of baffles','Length'],'Bell_summary'] = N_b,L_eff*1000
    calculations_df.loc[['Surface Area','Tube Heat transfer Coef.','Shell Heat transfer Coef.','Uclean','Udirty','Uservice','Shell Pressure Drop','Tube Pressure Drop','Shell Reynolds Number','Tube Reynolds Number','Tube Velocity','Shell Velocity','Over Design','Over Surface'],'Bell_summary']= A_available,h_t_i,h_s_i,U_clean,U_dirty,U_required,total_dp_shell,total_dp_tube,Re_s,Re_t,v_t,v_w,OD,A_over
            
    return U_clean,U_dirty,U_required,OD,total_dp_shell,total_dp_tube


In [4]:
import pandas as pd
import openpyxl
import ht
import numpy as np

workbook= openpyxl.load_workbook('datasheet_bell_tiral.xlsx', data_only=True)
def load_data_table():
    url ='data_tables.csv'

    return pd.read_csv(url)
shell_side = 'Cold Side'
thickness_table = load_data_table().iloc[11:36,1:4]
thickness_table.columns = thickness_table.iloc[0]
thickness_table = thickness_table[1:]
worksheet = workbook['Sheet1']     
s3 = worksheet['I10'].value
t1_s = worksheet['H20'].value
t2_s =worksheet['I20'].value
m_s = worksheet['H14'].value
Cp_s = worksheet['H24'].value
mu_s = worksheet['H22'].value*0.001
rho_s =  worksheet['H21'].value
k_s = worksheet['H25'].value
fouling_s = worksheet['H30'].value
mu_t = worksheet['K22'].value*0.001
fouling_t =  worksheet['K30'].value
rho_t =  worksheet['K21'].value
m_t = worksheet['K14'].value
t1_t =  worksheet['K20'].value
t2_t = worksheet['L20'].value
Cp_t = worksheet['K24'].value
k_t =worksheet['K25'].value
shell_side ='Cold Side'
s2 = 'Cold side T2'

Shell_list = [m_s, t1_s, t2_s, rho_s, Cp_s, mu_s, k_s, fouling_s]
Tube_list = [m_t, t1_t, t2_t, rho_t, Cp_t, mu_t, k_t, fouling_t]

HB_data = Heat_balance(shell_side, Tube_list, Shell_list,s2,s3)
Q, dTlm, ft = HB_data[0], HB_data[1], HB_data[2]

Do = worksheet['F42'].value
thick = float(thickness_table[thickness_table['Gauge']==str(worksheet['H42'].value)]['mm']) #2.108
print(worksheet['H42'].value)
print(thick)
Di = (Do - 2*thick)*0.001

shell_D = worksheet['H44'].value/1000
tn = worksheet['D42'].value

#Do = float(st.session_state.geo_df.iloc[2,1])
#Di = (Do - 2*float(st.session_state.geo_df.iloc[3,1]))*0.001
L = worksheet['J42'].value
tpitch = Do*worksheet['M43'].value
b_space = worksheet['M48'].value
b_cut = worksheet['I48'].value


print(tn,shell_D,b_cut,b_space,tpitch,b_cut)

pn = worksheet['H37'].value

pitch =worksheet['M42'].value
geo_list = [tn ,pn,Do, Di, pitch, tpitch,L, b_space, b_cut,shell_D]
print(geo_list,Shell_list,Tube_list)


14
2.11
414 0.609 25 234 23.8125 25
[414, 2, 19.05, 0.014830000000000003, 'triangle 30', 23.8125, 4267, 234, 25, 0.609] [140000, 40, 60, 716, 0.51, 0.00062, 0.1276, 0.00043103448275862074] [120000, 104, 85.11111, 578, 0.63, 0.00016, 0.093, 0.0003448275862068966]


In [5]:
para_input_df.loc[:,'Kern_summary'] = para_input_df.loc[:,'Bell_summary'] = [m_t, t1_t, t2_t, rho_t, Cp_t, mu_t*1000, k_t, fouling_t,m_s, t1_s, t2_s, rho_s, Cp_s, mu_s*1000, k_s, fouling_s]
calculations_df.loc[['Duty','LMTD','Ft','Corrected LMTD'],'Kern_summary'] = calculations_df.loc[['Duty','LMTD','Ft','Corrected LMTD'],'Bell_summary'] = Q,dTlm,ft,dTlm*ft
print(Tube_list, Shell_list, geo_list)
geo_input_df.loc[['Shell D','Baffle Spacing','Do','Di','Length','Number of tubes','Number of passes','Tube pitch','pitch type'],'Kern_summary']=geo_input_df.loc[['Shell D','Baffle Spacing','Do','Di','Length','Number of tubes','Number of passes','Tube pitch','pitch type'],'Bell_summary']=shell_D,b_space,Do,Di*1000,L,tn,pn,tpitch,pitch
dp_s, dp_t, h_shell, h_t, Uc, Ud, U_calc, Rdesign, Rsevice = kern(Tube_list, Shell_list, geo_list,s3,HB_data,geo_input_df,calculations_df)
geo_list = [tn ,pn,Do, Di, pitch, tpitch,L, b_space, b_cut,shell_D]
Shell_list = [m_s, t1_s, t2_s, rho_s, Cp_s, mu_s*1000, k_s, fouling_s]
Tube_list = [m_t, t1_t, t2_t, rho_t, Cp_t, mu_t*1000, k_t, fouling_t]
print(Tube_list, Shell_list, geo_list)
U_clean,U_dirty,U_required,OD,total_dp_shell,total_dp_tube=bell_delaware(Tube_list, Shell_list ,h_t,h_shell,geo_list,s3,HB_data,geo_input_df,calculations_df)
print(U_clean,U_dirty,U_required,OD,total_dp_shell,total_dp_tube)           

[120000, 104, 85.11111, 578, 0.63, 0.00016, 0.093, 0.0003448275862068966] [140000, 40, 60, 716, 0.51, 0.00062, 0.1276, 0.00043103448275862074] [414, 2, 19.05, 0.014830000000000003, 'triangle 30', 23.8125, 4267, 234, 25, 0.609]
[120000, 104, 85.11111, 578, 0.63, 0.16, 0.093, 0.0003448275862068966] [140000, 40, 60, 716, 0.51, 0.62, 0.1276, 0.00043103448275862074] [414, 2, 19.05, 0.014830000000000003, 'triangle 30', 23.8125, 4267, 234, 25, 0.609]
dp for T_wall 85.69508437673325
dp for Re_s 39151.534895259065
dp for j 0.00531666065690833
2637.4944534573197
value for N_b 16
1403.2871913523024
value for v_t 1.6129034764102101
value for Re_t 86408.68278052784
value for Pr_t 4.526245161290323
value for f_turbulent 0.004640564446301345
value for Nu_tube 425.61403618503465
value for h_t_i 2300.9106288050907
dp for U_clean 750.4441770041726
dp for U_dirty 447.9194728522786
dp for G_w 1176.2224961278055
dp for cent baffle 0.27886218145069813
dp for baffle wind 0.22942536249103013
dp for entrance 0

In [6]:
calculations_df

,Kern_summary,Bell_summary
Duty,1.660764e+06,1.660764e+06
LMTD,5.165268e+01,5.165268e+01
Ft,9.938095e-01,9.938095e-01
Corrected LMTD,5.133292e+01,5.133292e+01
Surface Area,1.057226e+02,1.027048e+02
Tube Heat transfer Coef.,2.730116e+03,2.300911e+03
Shell Heat transfer Coef.,1.841018e+03,2.637494e+03
Uclean,9.492663e+02,7.504442e+02
Udirty,5.188254e+02,4.479195e+02
Uservice,3.060160e+02,2.708579e+02


In [7]:
# input data
para_input_df

,Kern_summary,Bell_summary
Tube Flow rate,120000.000000,120000.000000
Tube inlet temperature,104.000000,104.000000
Tube outlet temperature,85.111110,85.111110
Tube Density,578.000000,578.000000
Tube Heat Capacity,0.630000,0.630000
Tube Viscosity,0.160000,0.160000
Tube Thermal conductivity,0.093000,0.093000
Tube Fouling factor,0.000345,0.000345
Shell flow rate,140000.000000,140000.000000
Shell inlet temperature,40.000000,40.000000


In [8]:


L = 5000

# initialise: 
# 1. Shell side friction factor 2. Heat transfer j-factor
f_s_i = 0.1029
j_s_i = 0.0053

#initialise baffle cut
b_cut = 25
#initialise shell diameter
shell_D = 590
pn = 2
def initialise(f_s_i,j_s_i,b_cut,shell_D, pn, L):
        mu_s = worksheet['H22'].value
        mu_t = worksheet['K22'].value
        Do = 19.05
        Di = Do- 2*2.11
        t_p_angle = 30
        shell_D = shell_D
        b_cut= b_cut
        L= L/1000
        L_tb = 0.4 # Diametral Tube-Baffle Clearance
        D_sb = 3.1+0.004*shell_D
        L_s = 0.1*shell_D
        L_eff = L-2*L_s/1000 # Effective tube length
        #Estimate: tube count and baffle spacing
        pitch = 23.81 /1000
        t_p = 23.81
        if t_p_angle == 45:
            t_p_effective = t_p/(2**0.5)
        else: t_p_effective = t_p
        if t_p_angle == 30:
            t_arrg = np.sqrt(3)/2
        elif t_p_angle == 45:
            t_arrg = 1/np.sqrt(2)
        else : t_arrg = 1
        
        DBundle = shell_D/1000 - 2*ht.shell_clearance(DShell=shell_D/1000)
        tn = ht.hx.Ntubes(DBundle, Do/1000, pitch, Ntp=2, angle=t_p_angle, Method=None)
        print(tn)
        D_otl = shell_D - (12.5+(shell_D/200))

        theta_Ds = 2*np.arccos(1-(2*b_cut)/100) # Baffle window angle
        theta_CTL = 2*np.arccos(shell_D*(1-2*b_cut/100)/(D_otl-Do)) #b_cut baffle cut, shell_D isnide shell diameter mm
        F_w = (theta_CTL-np.sin(theta_CTL))/(2*np.pi)
        S_wg = (((shell_D/1000)**2)/8)*(theta_Ds-np.sin(theta_Ds)) #Gross window area
        S_wt = tn*F_w*np.pi*((Do/1000)**2)/4 #Window area occupied with tubes
        S_w = S_wg - S_wt # Net Cross flow area
        S_m = S_w
        G_w = (m_s/3600)/np.sqrt(S_m*S_w)
        v_w = G_w/rho_s

        #S_m = (Lb_cut/1000)*((shell_D-D_otl)+(D_otl-Do)*(t_p-Do)/t_p_effective)/1000
        Lb_cut = LB_in = LB_out = ((S_m*1000)/((shell_D-D_otl)+(D_otl-Do)*(t_p-Do)/t_p_effective))*1000
        Lb_cut

        # Calculate shell side correction factors
        Gs = (m_s/3600)/S_m
        Re_s = (Do/1000)*Gs/(mu_s*0.001)
        print('dp for Re_s '+str(Re_s))
        Pr_s = (mu_s/1000)*Cp_s/k_s*3600
        # j- ideal factor coefficients
        a1 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['a_{1}'],0).sum()
        a2 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['a_{2}'],0).sum()
        a3 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['a_{3}'],0).sum()
        a4 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['a_{4}'],0).sum()
        a=a3/(1+0.14*((Re_s)**a4))
        #print(a)
        #print(Re_s)
        j=a1*((1.33/(t_p/Do))**a)*(Re_s**a2)
        print('value for j '+str(j))
        #print(j)
        mu_s_w = mu_s
        h_s_i = Cp_s*(m_s/S_m)*j*(Pr_s**(-2/3))*((mu_s/mu_s_w)**0.14)
        print(h_s_i)

        # 1. correction factor for baffle window flow
        N_b = 1 +int((L-(2*L_s*0.001)-(LB_in+LB_out)*0.001)/(Lb_cut*0.001)) # number of baffles
        theta_CTL = 2*np.arccos(shell_D*(1-2*b_cut/100)/(D_otl-Do)) #b_cut baffle cut, shell_D isnide shell diameter mm
        F_w = (theta_CTL-np.sin(theta_CTL))/(2*np.pi)

        F_c = 1-2*F_w # Fraction of tubes in cross flow
        j_c = 0.55+0.72*F_c

        # 2. correction factor for baffle leakage
        theta_Ds = 2*np.arccos(1-(2*b_cut)/100) # Baffle window angle
        S_sb = (shell_D/1000)*(D_sb/1000)*(np.pi-0.5*theta_Ds)    # Shell to baffle leakage area
        S_tb = (np.pi/4)*((((Do/1000)+(L_tb/1000))**2)-((Do/1000)**2))*tn*(1-F_w)  # Tube to baffle leakage area

        r_L = (S_sb+S_tb)/S_m # ratio of leakage to cross flow
        r_S = S_sb/(S_sb+S_tb) # ratio of shell-baffle to total area

        j_l = 0.44*(1-r_S)+(1-0.44*(1-r_S))*np.exp(-2.2*r_L) # Baffle leakage correction factor

        # 3. correction factor for bundle bypass
            # number_of_sealing_strips   
        P_p = t_p * t_arrg # Tube row distance in flow direction
        N_TCC = (shell_D/P_p)*(1-(2*b_cut)/100) #N_TCC number of tube rows between baffle
        N_ss = int(N_TCC/6)
        r_ss = N_ss/N_TCC # Nss Number of sealing strips
        print('N_ss is '+str(N_ss)+' While N_Tcc ia '+str(N_TCC))
        S_b = (Lb_cut/1000)*(shell_D-D_otl-(Do/2))/1000 #bundle pybass area
        if Re_s < 100:
            C_j = 1.35
        else: C_j = 1.25 # Correlation constant

        if r_ss >= 0.5:
            j_b = 1
        else: j_b = np.exp(-1*C_j*(S_b/S_m)*(1-((2*r_ss)**(1/3))))

        # 4. Correction factor for adverse temperature gradient
        N_tcw = (0.8/P_p)*(shell_D*b_cut/100-(shell_D-(D_otl-Do))/2)
        N_b = 1 +int((L-(2*L_s*0.001)-(LB_in+LB_out)*0.001)/(Lb_cut*0.001)) # number of baffles
        print('value for N_b '+str(N_b))
        N_c = (N_tcw +N_TCC)*(1+N_b) # tube rows crossed in entire exchanger
        j_RL = (10/N_c)**0.18
        if Re_s <= 20:
            j_R = j_RL
        elif Re_s <100:
            j_R = j_RL+((20-Re_s)/80)*(j_RL-1)
        else: j_R = 1

        # 5. correction factor for unequal baffle spacing
        if Re_s < 100:
            n1 = 1/3
        else: n1 = 0.6
        j_s =((N_b-1)+(LB_in/Lb_cut)**(1-n1)+(LB_out/Lb_cut)**(1-n1))/((N_b-1)+(LB_in/Lb_cut)+(LB_out/Lb_cut))


        h_shell = j_s * j_R *j_b * j_l * j_c * h_s_i
        print(h_shell)
        ### Shell side pressure drop
        b1 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{1}'],0).sum()
        b2 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{2}'],0).sum()
        b3 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{3}'],0).sum()
        b4 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{4}'],0).sum()
        b = b3/(1+0.14*(Re_s**b4))
        f_s = b1*((1.33/(t_p/Do))**b)*(Re_s**b2)
        dp_shell_ideal = 2*f_s*(Gs**2)*N_TCC*(mu_s_w/mu_s)**0.14/(rho_s)/100000 #N_TCC number of tube rows between baffle
        #print(dp_shell_ideal)

        # 1. correction factor for baffle leakage
        f_p = 0.8-0.15*(1+r_S) # r_s ratio of shell-baffle to total area
        R_L = np.exp(-1.33*(1+r_S)*(r_L**f_p))# r_l ratio of  leakage area to cross flow

        # pressure drop for an ideal window section
        S_wg = (((shell_D/1000)**2)/8)*(theta_Ds-np.sin(theta_Ds)) #Gross window area
        S_wt = tn*F_w*np.pi*((Do/1000)**2)/4 #Window area occupied with tubes
        S_w = S_wg - S_wt # Net Cross flow area
        G_w = (m_s/3600)/np.sqrt(S_m*S_w)
        v_w = G_w/rho_s
        print('dp for G_w '+str(G_w))
        D_w = 4*S_w/(np.pi*(Do/1000)*tn*F_w+(shell_D/1000)*theta_Ds)

        #pressure drop for turbluent flow in ideal window section
        if Re_s >= 100:
            dp_window = N_b*R_L*(2+0.6*N_tcw)*(G_w**2)/(2*rho_s)/100000

            
        else:
            dp_window = N_b*R_L*((26*G_w*Cp_s/rho_s)*(N_tcw/(tpitch-Do)/1000 + (Lb_cut/1000)/(D_w**2)) + (G_w**2)/rho_s)/100000
            

        # 2. Correction factor for bundle bypass effect
        if Re_s < 100:
            C_r = 4.5
        else: C_r = 3.7

        if r_ss >= 0.5:
            R_b = 1
        else:
            R_b = np.exp(-1*C_r*(S_b/S_m)*(1-(2*r_ss)**(1/3)))

        # 3. Correction for unequal baffle spacing inlet/ outlet
        if Re_s < 100:
            n = 1 # Slope of friction factor curve
        else: n = 0.2
        R_s = (Lb_cut/LB_in)**(2-n) + (Lb_cut/LB_out)**(2-n) # correction factor

        # Shell side pressure drop (Excluding nozzles)
        dp_cent_baff = dp_shell_ideal*(N_b-1)*R_L*R_b # pressure drop in baffle windows
        dp_baff_window = dp_window # pressure drop in baffle windows
        dp_entrance_exit = dp_shell_ideal*R_s*R_b*(1+N_tcw/N_TCC) # pressure drop in entrance / exit baffles
        print('dp for cent baffle '+str(dp_cent_baff))
        print('dp for baffle wind '+str(dp_baff_window))
        print('dp for entrance '+str(dp_entrance_exit))

        total_dp_shell = dp_cent_baff + dp_baff_window + dp_entrance_exit
        total_dp_shell
        A_available = L_eff * tn *np.pi*(Do/1000)
        ### Tube Side Heat transfer coeficient
        a_tube = (np.pi*((Di/1000)**2)*tn)/(4*pn) # Flow area
        v_t = (m_t/3600)/(rho_t*a_tube) #velocity through tube
        print('value for v_t '+str(v_t))
        Re_t = (Di/1000)*rho_t*v_t/(mu_t/1000)
        print('value for Re_t '+str(Re_t))
        Pr_t = Cp_t*(mu_t/1000)/k_t*3600
        print('value for Pr_t '+str(Pr_t))
        L_eff = L-2*L_s/1000 # Effective tube length

        # Nusselt Number Calculation
        Nu_laminar = 1.86*(Re_t*Pr_t*(Di/1000)/L_eff)**(1/3)
        # Turbulent flow Petukhov-Kirillov
        f_turbulent = (1.58*np.log(Re_t)-3.28)**-2
        print('value for f_turbulent '+str(f_turbulent))
        Nu_turb = (f_turbulent/2)*Re_t*Pr_t/(1.07+12.7*((f_turbulent/2)**0.5)*((Pr_t**(2/3))-1))
        # Transition flow Nu
        Nu_2300 = 1.86*(2300*Pr_t*(Di/1000)/L_eff)**(1/3)

        f_Re_10000 = (1.58*np.log(10000)-3.28)**-2
        Nu_10000 = (f_Re_10000/2)*10000*Pr_t/(1.07+12.7*((f_Re_10000/2)**0.5)*((Pr_t**(2/3))-1))

        Nu_trans = Nu_2300+(Nu_10000-Nu_2300)*(Re_t-2300)/(10000-2300)


        if Re_t <= 2300:
            Nu_tube = Nu_laminar
        elif Re_t < 10000:
            Nu_tube = Nu_trans
        else: Nu_tube = Nu_turb
        print('value for Nu_tube '+str(Nu_tube))
        mu_t_w = mu_t
        h_t_i=Nu_tube*k_t/(Di/1000)*(mu_t/mu_t_w)**0.14
        print('value for h_t_i '+str(h_t_i))
        total_dp_tube =((4*f_turbulent*L*pn/(Di/1000))+4*pn)*rho_t*(v_t**2)/2/100000
        print(total_dp_tube,L)
        return [total_dp_shell, h_shell , A_available,h_t_i,total_dp_tube]
sol1 = initialise(f_s_i,j_s_i,25,590,2,5000)
sol2 = initialise(f_s_i,j_s_i,25,590,2,4000)

476
dp for Re_s 40887.85441053393
value for j 0.005227625651851436
2969.334062523191
N_ss is 2 While N_Tcc ia 14.306453541448098
value for N_b 20
1551.3953146841611
dp for G_w 1330.7333193979546
dp for cent baffle 0.37136448961590107
dp for baffle wind 0.3568709213828771
dp for entrance 0.18447760066177799
value for v_t 1.402819410155099
value for Re_t 75153.77031751792
value for Pr_t 3.901935483870968
value for f_turbulent 0.004783174614577113
value for Nu_tube 352.73168208201906
value for h_t_i 2212.005828295871
0.11887093066861441 5.0
476
dp for Re_s 40887.85441053393
value for j 0.005227625651851436
2969.334062523191
N_ss is 2 While N_Tcc ia 14.306453541448098
value for N_b 15
1551.3953146841611
dp for G_w 1330.7333193979546
dp for cent baffle 0.2736369923485587
dp for baffle wind 0.26765319103715784
dp for entrance 0.18447760066177799
value for v_t 1.402819410155099
value for Re_t 75153.77031751792
value for Pr_t 3.901935483870968
value for f_turbulent 0.004783174614577113
value f

In [9]:
dps1_hs1 = sol1[0]/(sol1[1]**2)
dps2_hs2 = sol2[0]/(sol2[1]**2)
A1,A2 = sol1[2],sol2[2]
k1 = (dps2_hs2-dps1_hs1)/(A2-A1)
k2 = dps1_hs1-(k1*A1)
print(k1,k2)

2.7265761178636237e-09 1.9282833973087812e-11


In [10]:
print((k1*A1+k2)*(sol1[1]**2),sol1[0])
print((k1*A2+k2)*(sol2[1]**2),sol2[0])
k3 = sol1[4]/(sol1[2]*sol1[3]**3.5)
print(k3,k3*(sol2[2]*sol2[3]**3.5))
sol2[4]

0.9127130116605562 0.9127130116605562
0.7257677840474943 0.7257677840474945
1.6790906688304606e-15 0.09452211242432636


0.10419632475856808

In [11]:
HB_data = Heat_balance(shell_side, Tube_list, Shell_list,s2,s3)
Q, dTlm, ft = HB_data[0], HB_data[1], HB_data[2]
print(HB_data)
c1 =Q/(dTlm*ft)
c2=Q/(dTlm*ft)*(fouling_s+fouling_t*((Do*0.001)/Di))
c3 = c1*((Do*0.001)/Di)
c1/2969+c2+c3/1550

[1660764.0976920004, 51.65267942244251, 0.9938094866510533]


65.9850582494764

In [12]:
import numpy as np
from scipy.optimize import fsolve
# for a given variable w, this function returns F(w)
# if w is the solution of the nonlinear system, then 
# F(w)=0
# F can be interpreted as the residual

def nonlinearEquation(w):
    # A = w[2], h_s is w[0] and h_t is w[1]
    F=np.zeros(3)
    F[0]=(c1/w[0])+(c3/w[1])-w[2]+c2
    F[1]=k3* w[2]* np.sign(w[1]) * ((np.abs(w[1])) ** (3.5)) - 0.2
    F[2]=k1*w[2]*(w[0]**2)+k2*(w[0]**2)-0.5
    return F
# generate an initial guess
initialGuess=np.array([sol1[1],sol1[3],sol1[2]])   
 
# solve the problem    
solutionInfo=fsolve(nonlinearEquation,initialGuess,full_output=1)
solutionInfo

(array([1757.15015785, 3272.91800368,   59.38584081]),
 {'nfev': 15,
  'fjac': array([[-0.99895863, -0.0012657 ,  0.04560752],
         [ 0.03800576, -0.57614212,  0.81646544],
         [-0.02524301, -0.81734855, -0.57559025]]),
  'r': array([ 1.13096512e-02,  4.87364982e-03,  1.08523484e+00, -3.51522000e-04,
         -4.00603573e-02,  2.42803782e-02]),
  'qtf': array([ 5.15463661e-09, -4.41694706e-10,  4.13156986e-10]),
  'fvec': array([-5.97673022e-11, -1.03705933e-12, -4.16666701e-12])},
 1,
 'The solution converged.')

In [13]:
h_t_i = solutionInfo[0][1]
Di = Do-2*2.11
mu_t = worksheet['K22'].value
Pr_t = Cp_t*(mu_t/1000)/k_t*3600
mu_t_w = mu_t
Nu_turb = h_t_i/(k_t/(Di/1000)*(mu_t/mu_t_w)**0.14)
def nonlinearEquation2(w):
    # A = w[2], h_s is w[0] and h_t is w[1]
    F=np.zeros(2)
   
    F[0]=np.exp((np.sqrt(1/w[0])+3.28)/1.58) - w[1]
    F[1]= (w[0]/2)*w[1]*Pr_t/(1.07+12.7*((w[0]/2)**0.5)*((Pr_t**(2/3))-1))-Nu_turb
    
    return F
# generate an initial guess
initialGuess=np.array([0.0046,86385])   
 
# solve the problem    
solutionInfo2=fsolve(nonlinearEquation2,initialGuess,full_output=1)


#f_turbulent = (1.58*np.log(Re_t)-3.28)**-2
#Nu_turb = (solutionInfo[0][0]/2)*solutionInfo[0][1]*Pr_t/(1.07+12.7*((solutionInfo[0][0]/2)**0.5)*((Pr_t**(2/3))-1))
print(solutionInfo2)
Re_t = solutionInfo2[0][1] 
v_t = Re_t/((Di/1000)*rho_t/(mu_t/1000))
print(v_t)
#   a_tube = (np.pi*((Di/1000)**2)*tn)/(4*pn)
a_tube = (m_t/3600)/(v_t*rho_t)
tn = a_tube*4*pn/(np.pi*((Di/1000)**2))
print(tn)
L_s = 0.1*shell_D
A_available = solutionInfo[0][2]
L_eff = A_available/tn *np.pi*(Do/1000)
L = L_eff+2*L_s/1000
t_p = 23.81
t_p_angle = 0
if t_p_angle == 45:
    t_p_effective = t_p/(2**0.5)
else: t_p_effective = t_p
D_otl = shell_D - (12.5+(shell_D/200))

theta_Ds = 2*np.arccos(1-(2*b_cut)/100) # Baffle window angle
theta_CTL = 2*np.arccos(shell_D*(1-2*b_cut/100)/(D_otl-Do)) #b_cut baffle cut, shell_D isnide shell diameter mm
F_w = (theta_CTL-np.sin(theta_CTL))/(2*np.pi)
S_wg = (((shell_D/1000)**2)/8)*(theta_Ds-np.sin(theta_Ds)) #Gross window area
S_wt = tn*F_w*np.pi*((Do/1000)**2)/4 #Window area occupied with tubes
S_w = S_wg - S_wt # Net Cross flow area
S_m = S_w
G_w = (m_s/3600)/np.sqrt(S_m*S_w)
v_w = G_w/rho_s

#S_m = (Lb_cut/1000)*((shell_D-D_otl)+(D_otl-Do)*(t_p-Do)/t_p_effective)/1000
Lb_cut = LB_in = LB_out = ((S_m*1000)/((shell_D-D_otl)+(D_otl-Do)*(t_p-Do)/t_p_effective))*1000
print(Lb_cut)
N_b = 1 +int((L-(2*L_s*0.001)-(LB_in+LB_out)*0.001)/(Lb_cut*0.001)) # number of baffles
print(N_b,L_s)
ht.hx.DBundle_for_Ntubes_Phadkeb(tn, Do/1000, t_p/1000, pn, angle=30)*1000

(array([4.32820871e-03, 1.20120876e+05]), {'nfev': 13, 'fjac': array([[-9.99999743e-01,  7.17519568e-04],
       [-7.17519568e-04, -9.99999743e-01]]), 'r': array([ 1.30299188e+08,  8.56576980e-01, -3.73056243e-03]), 'qtf': array([3.18363806e-05, 2.32792211e-08]), 'fvec': array([1.73458830e-08, 1.13686838e-13])}, 1, 'The solution converged.')
2.242174877561744
297.8099727707074
302.6998835691062
0 59.0


473.31584771529333

In [21]:
import numpy as np
from scipy.optimize import fsolve

L = 5000

# initialise: 
# 1. Shell side friction factor 2. Heat transfer j-factor
f_s_i = 0.1029
j_s_i = 0.0053

#initialise baffle cut
b_cut = 25
#initialise shell diameter
shell_D = 590
pn = 2
error =2
def initialise(f_s_i,j_s_i,b_cut,shell_D, pn, L):
        mu_s = worksheet['H22'].value
        mu_t = worksheet['K22'].value
        Do = 19.05
        Di = Do- 2*2.11
        t_p_angle = 30
        shell_D = shell_D
        b_cut= b_cut
        L= L/1000
        L_tb = 0.4 # Diametral Tube-Baffle Clearance
        D_sb = 3.1+0.004*shell_D
        L_s = 0.1*shell_D
        L_eff = L-2*L_s/1000 # Effective tube length
        #Estimate: tube count and baffle spacing
        pitch = 23.81 /1000
        t_p = 23.81
        if t_p_angle == 45:
            t_p_effective = t_p/(2**0.5)
        else: t_p_effective = t_p
        if t_p_angle == 30:
            t_arrg = np.sqrt(3)/2
        elif t_p_angle == 45:
            t_arrg = 1/np.sqrt(2)
        else : t_arrg = 1
        
        DBundle = shell_D/1000 - 2*ht.shell_clearance(DShell=shell_D/1000)
        tn = ht.hx.Ntubes(DBundle, Do/1000, pitch, Ntp=2, angle=t_p_angle, Method=None)
        print(tn)
        D_otl = shell_D - (12.5+(shell_D/200))

        theta_Ds = 2*np.arccos(1-(2*b_cut)/100) # Baffle window angle
        theta_CTL = 2*np.arccos(shell_D*(1-2*b_cut/100)/(D_otl-Do)) #b_cut baffle cut, shell_D isnide shell diameter mm
        F_w = (theta_CTL-np.sin(theta_CTL))/(2*np.pi)
        S_wg = (((shell_D/1000)**2)/8)*(theta_Ds-np.sin(theta_Ds)) #Gross window area
        S_wt = tn*F_w*np.pi*((Do/1000)**2)/4 #Window area occupied with tubes
        S_w = S_wg - S_wt # Net Cross flow area
        S_m = S_w
        G_w = (m_s/3600)/np.sqrt(S_m*S_w)
        v_w = G_w/rho_s

        #S_m = (Lb_cut/1000)*((shell_D-D_otl)+(D_otl-Do)*(t_p-Do)/t_p_effective)/1000
        Lb_cut = LB_in = LB_out = ((S_m*1000)/((shell_D-D_otl)+(D_otl-Do)*(t_p-Do)/t_p_effective))*1000
        Lb_cut

        # Calculate shell side correction factors
        Gs = (m_s/3600)/S_m
        Re_s = (Do/1000)*Gs/(mu_s*0.001)
        print('dp for Re_s '+str(Re_s))
        Pr_s = (mu_s/1000)*Cp_s/k_s*3600
        # j- ideal factor coefficients
        a1 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['a_{1}'],0).sum()
        a2 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['a_{2}'],0).sum()
        a3 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['a_{3}'],0).sum()
        a4 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['a_{4}'],0).sum()
        a=a3/(1+0.14*((Re_s)**a4))
        #print(a)
        #print(Re_s)
        j=a1*((1.33/(t_p/Do))**a)*(Re_s**a2)
        print('value for j '+str(j))
        #print(j)
        mu_s_w = mu_s
        h_s_i = Cp_s*(m_s/S_m)*j*(Pr_s**(-2/3))*((mu_s/mu_s_w)**0.14)
        print(h_s_i)

        # 1. correction factor for baffle window flow
        N_b = 1 +int((L-(2*L_s*0.001)-(LB_in+LB_out)*0.001)/(Lb_cut*0.001)) # number of baffles
        theta_CTL = 2*np.arccos(shell_D*(1-2*b_cut/100)/(D_otl-Do)) #b_cut baffle cut, shell_D isnide shell diameter mm
        F_w = (theta_CTL-np.sin(theta_CTL))/(2*np.pi)

        F_c = 1-2*F_w # Fraction of tubes in cross flow
        j_c = 0.55+0.72*F_c

        # 2. correction factor for baffle leakage
        theta_Ds = 2*np.arccos(1-(2*b_cut)/100) # Baffle window angle
        S_sb = (shell_D/1000)*(D_sb/1000)*(np.pi-0.5*theta_Ds)    # Shell to baffle leakage area
        S_tb = (np.pi/4)*((((Do/1000)+(L_tb/1000))**2)-((Do/1000)**2))*tn*(1-F_w)  # Tube to baffle leakage area

        r_L = (S_sb+S_tb)/S_m # ratio of leakage to cross flow
        r_S = S_sb/(S_sb+S_tb) # ratio of shell-baffle to total area

        j_l = 0.44*(1-r_S)+(1-0.44*(1-r_S))*np.exp(-2.2*r_L) # Baffle leakage correction factor

        # 3. correction factor for bundle bypass
            # number_of_sealing_strips   
        P_p = t_p * t_arrg # Tube row distance in flow direction
        N_TCC = (shell_D/P_p)*(1-(2*b_cut)/100) #N_TCC number of tube rows between baffle
        N_ss = int(N_TCC/6)
        r_ss = N_ss/N_TCC # Nss Number of sealing strips
        print('N_ss is '+str(N_ss)+' While N_Tcc ia '+str(N_TCC))
        S_b = (Lb_cut/1000)*(shell_D-D_otl-(Do/2))/1000 #bundle pybass area
        if Re_s < 100:
            C_j = 1.35
        else: C_j = 1.25 # Correlation constant

        if r_ss >= 0.5:
            j_b = 1
        else: j_b = np.exp(-1*C_j*(S_b/S_m)*(1-((2*r_ss)**(1/3))))

        # 4. Correction factor for adverse temperature gradient
        N_tcw = (0.8/P_p)*(shell_D*b_cut/100-(shell_D-(D_otl-Do))/2)
        N_b = 1 +int((L-(2*L_s*0.001)-(LB_in+LB_out)*0.001)/(Lb_cut*0.001)) # number of baffles
        print('value for N_b '+str(N_b))
        N_c = (N_tcw +N_TCC)*(1+N_b) # tube rows crossed in entire exchanger
        j_RL = (10/N_c)**0.18
        if Re_s <= 20:
            j_R = j_RL
        elif Re_s <100:
            j_R = j_RL+((20-Re_s)/80)*(j_RL-1)
        else: j_R = 1

        # 5. correction factor for unequal baffle spacing
        if Re_s < 100:
            n1 = 1/3
        else: n1 = 0.6
        j_s =((N_b-1)+(LB_in/Lb_cut)**(1-n1)+(LB_out/Lb_cut)**(1-n1))/((N_b-1)+(LB_in/Lb_cut)+(LB_out/Lb_cut))


        h_shell = j_s * j_R *j_b * j_l * j_c * h_s_i
        print(h_shell)
        ### Shell side pressure drop
        b1 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{1}'],0).sum()
        b2 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{2}'],0).sum()
        b3 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{3}'],0).sum()
        b4 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{4}'],0).sum()
        b = b3/(1+0.14*(Re_s**b4))
        f_s = b1*((1.33/(t_p/Do))**b)*(Re_s**b2)
        dp_shell_ideal = 2*f_s*(Gs**2)*N_TCC*(mu_s_w/mu_s)**0.14/(rho_s)/100000 #N_TCC number of tube rows between baffle
        #print(dp_shell_ideal)

        # 1. correction factor for baffle leakage
        f_p = 0.8-0.15*(1+r_S) # r_s ratio of shell-baffle to total area
        R_L = np.exp(-1.33*(1+r_S)*(r_L**f_p))# r_l ratio of  leakage area to cross flow

        # pressure drop for an ideal window section
        S_wg = (((shell_D/1000)**2)/8)*(theta_Ds-np.sin(theta_Ds)) #Gross window area
        S_wt = tn*F_w*np.pi*((Do/1000)**2)/4 #Window area occupied with tubes
        S_w = S_wg - S_wt # Net Cross flow area
        G_w = (m_s/3600)/np.sqrt(S_m*S_w)
        v_w = G_w/rho_s
        print('dp for G_w '+str(G_w))
        D_w = 4*S_w/(np.pi*(Do/1000)*tn*F_w+(shell_D/1000)*theta_Ds)

        #pressure drop for turbluent flow in ideal window section
        if Re_s >= 100:
            dp_window = N_b*R_L*(2+0.6*N_tcw)*(G_w**2)/(2*rho_s)/100000

            
        else:
            dp_window = N_b*R_L*((26*G_w*Cp_s/rho_s)*(N_tcw/(tpitch-Do)/1000 + (Lb_cut/1000)/(D_w**2)) + (G_w**2)/rho_s)/100000
            

        # 2. Correction factor for bundle bypass effect
        if Re_s < 100:
            C_r = 4.5
        else: C_r = 3.7

        if r_ss >= 0.5:
            R_b = 1
        else:
            R_b = np.exp(-1*C_r*(S_b/S_m)*(1-(2*r_ss)**(1/3)))

        # 3. Correction for unequal baffle spacing inlet/ outlet
        if Re_s < 100:
            n = 1 # Slope of friction factor curve
        else: n = 0.2
        R_s = (Lb_cut/LB_in)**(2-n) + (Lb_cut/LB_out)**(2-n) # correction factor

        # Shell side pressure drop (Excluding nozzles)
        dp_cent_baff = dp_shell_ideal*(N_b-1)*R_L*R_b # pressure drop in baffle windows
        dp_baff_window = dp_window # pressure drop in baffle windows
        dp_entrance_exit = dp_shell_ideal*R_s*R_b*(1+N_tcw/N_TCC) # pressure drop in entrance / exit baffles
        print('dp for cent baffle '+str(dp_cent_baff))
        print('dp for baffle wind '+str(dp_baff_window))
        print('dp for entrance '+str(dp_entrance_exit))

        total_dp_shell = dp_cent_baff + dp_baff_window + dp_entrance_exit
        total_dp_shell
        A_available = L_eff * tn *np.pi*(Do/1000)
        ### Tube Side Heat transfer coeficient
        a_tube = (np.pi*((Di/1000)**2)*tn)/(4*pn) # Flow area
        v_t = (m_t/3600)/(rho_t*a_tube) #velocity through tube
        print('value for v_t '+str(v_t))
        Re_t = (Di/1000)*rho_t*v_t/(mu_t/1000)
        print('value for Re_t '+str(Re_t))
        Pr_t = Cp_t*(mu_t/1000)/k_t*3600
        print('value for Pr_t '+str(Pr_t))
        L_eff = L-2*L_s/1000 # Effective tube length

        # Nusselt Number Calculation
        Nu_laminar = 1.86*(Re_t*Pr_t*(Di/1000)/L_eff)**(1/3)
        # Turbulent flow Petukhov-Kirillov
        f_turbulent = (1.58*np.log(Re_t)-3.28)**-2
        print('value for f_turbulent '+str(f_turbulent))
        Nu_turb = (f_turbulent/2)*Re_t*Pr_t/(1.07+12.7*((f_turbulent/2)**0.5)*((Pr_t**(2/3))-1))
        # Transition flow Nu
        Nu_2300 = 1.86*(2300*Pr_t*(Di/1000)/L_eff)**(1/3)

        f_Re_10000 = (1.58*np.log(10000)-3.28)**-2
        Nu_10000 = (f_Re_10000/2)*10000*Pr_t/(1.07+12.7*((f_Re_10000/2)**0.5)*((Pr_t**(2/3))-1))

        Nu_trans = Nu_2300+(Nu_10000-Nu_2300)*(Re_t-2300)/(10000-2300)


        if Re_t <= 2300:
            Nu_tube = Nu_laminar
        elif Re_t < 10000:
            Nu_tube = Nu_trans
        else: Nu_tube = Nu_turb
        print('value for Nu_tube '+str(Nu_tube))
        mu_t_w = mu_t
        h_t_i=Nu_tube*k_t/(Di/1000)*(mu_t/mu_t_w)**0.14
        print('value for h_t_i '+str(h_t_i))
        total_dp_tube =((4*f_turbulent*L*pn/(Di/1000))+4*pn)*rho_t*(v_t**2)/2/100000
        print(total_dp_tube,L)
        return [total_dp_shell, h_shell , A_available,h_t_i,total_dp_tube]
while error > 1 :
        sol1 = initialise(f_s_i,j_s_i,25,590,2,5000)
        sol2 = initialise(f_s_i,j_s_i,25,590,2,4000)
        dps1_hs1 = sol1[0]/(sol1[1]**2)
        dps2_hs2 = sol2[0]/(sol2[1]**2)
        A1,A2 = sol1[2],sol2[2]
        k1 = (dps2_hs2-dps1_hs1)/(A2-A1)
        k2 = dps1_hs1-(k1*A1)
        print(k1,k2)
        print((k1*A1+k2)*(sol1[1]**2),sol1[0])
        print((k1*A2+k2)*(sol2[1]**2),sol2[0])
        k3 = sol1[4]/(sol1[2]*sol1[3]**3.5)
        print(k3,k3*(sol2[2]*sol2[3]**3.5))
        sol2[4]
        HB_data = Heat_balance(shell_side, Tube_list, Shell_list,s2,s3)
        Q, dTlm, ft = HB_data[0], HB_data[1], HB_data[2]
        print(HB_data)
        c1 =Q/(dTlm*ft)
        c2=Q/(dTlm*ft)*(fouling_s+fouling_t*((Do*0.001)/Di))
        c3 = c1*((Do*0.001)/Di)
        c1/2969+c2+c3/1550

        # for a given variable w, this function returns F(w)
        # if w is the solution of the nonlinear system, then 
        # F(w)=0
        # F can be interpreted as the residual

        def nonlinearEquation(w):
            # A = w[2], h_s is w[0] and h_t is w[1]
            F=np.zeros(3)
            F[0]=(c1/w[0])+(c3/w[1])-w[2]+c2
            F[1]=k3* w[2]* np.sign(w[1]) * ((np.abs(w[1])) ** (3.5)) - 0.2
            F[2]=k1*w[2]*(w[0]**2)+k2*(w[0]**2)-0.5
            return F
        # generate an initial guess
        initialGuess=np.array([sol1[1],sol1[3],sol1[2]])   
        
        # solve the problem    
        solutionInfo=fsolve(nonlinearEquation,initialGuess,full_output=1)
        solutionInfo
        h_t_i = solutionInfo[0][1]
        Di = Do-2*2.11
        mu_t = worksheet['K22'].value
        Pr_t = Cp_t*(mu_t/1000)/k_t*3600
        mu_t_w = mu_t
        Nu_turb = h_t_i/(k_t/(Di/1000)*(mu_t/mu_t_w)**0.14)
        def nonlinearEquation2(w):
            # A = w[2], h_s is w[0] and h_t is w[1]
            F=np.zeros(2)
        
            F[0]=np.exp((np.sqrt(1/w[0])+3.28)/1.58) - w[1]
            F[1]= (w[0]/2)*w[1]*Pr_t/(1.07+12.7*((w[0]/2)**0.5)*((Pr_t**(2/3))-1))-Nu_turb
            
            return F
        # generate an initial guess
        initialGuess=np.array([0.0046,86385])   
        
        # solve the problem    
        solutionInfo2=fsolve(nonlinearEquation2,initialGuess,full_output=1)


        #f_turbulent = (1.58*np.log(Re_t)-3.28)**-2
        #Nu_turb = (solutionInfo[0][0]/2)*solutionInfo[0][1]*Pr_t/(1.07+12.7*((solutionInfo[0][0]/2)**0.5)*((Pr_t**(2/3))-1))
        print(solutionInfo2)
        Re_t = solutionInfo2[0][1] 
        v_t = Re_t/((Di/1000)*rho_t/(mu_t/1000))
        print(v_t)
        #   a_tube = (np.pi*((Di/1000)**2)*tn)/(4*pn)
        a_tube = (m_t/3600)/(v_t*rho_t)
        tn = a_tube*4*pn/(np.pi*((Di/1000)**2))
        print(tn)
        L_s = 0.1*shell_D
        A_available = solutionInfo[0][2]
        L_eff = A_available/tn *np.pi*(Do/1000)
        L = L_eff+2*L_s/1000
        t_p = 23.81
        t_p_angle = 0
        if t_p_angle == 45:
            t_p_effective = t_p/(2**0.5)
        else: t_p_effective = t_p
        D_otl = shell_D - (12.5+(shell_D/200))

        theta_Ds = 2*np.arccos(1-(2*b_cut)/100) # Baffle window angle
        theta_CTL = 2*np.arccos(shell_D*(1-2*b_cut/100)/(D_otl-Do)) #b_cut baffle cut, shell_D isnide shell diameter mm
        F_w = (theta_CTL-np.sin(theta_CTL))/(2*np.pi)
        S_wg = (((shell_D/1000)**2)/8)*(theta_Ds-np.sin(theta_Ds)) #Gross window area
        S_wt = tn*F_w*np.pi*((Do/1000)**2)/4 #Window area occupied with tubes
        S_w = S_wg - S_wt # Net Cross flow area
        S_m = S_w
        G_w = (m_s/3600)/np.sqrt(S_m*S_w)
        v_w = G_w/rho_s

        #S_m = (Lb_cut/1000)*((shell_D-D_otl)+(D_otl-Do)*(t_p-Do)/t_p_effective)/1000
        Lb_cut = LB_in = LB_out = ((S_m*1000)/((shell_D-D_otl)+(D_otl-Do)*(t_p-Do)/t_p_effective))*1000
        print(Lb_cut)
        N_b = 1 +int((L-(2*L_s*0.001)-(LB_in+LB_out)*0.001)/(Lb_cut*0.001)) # number of baffles
        print(N_b,L_s)
        new_shell_D = ht.hx.DBundle_for_Ntubes_Phadkeb(tn, Do/1000, t_p/1000, pn, angle=30)*1000+3.1+0.004*shell_D
        error = abs(shell_D-new_shell_D)
        shell_D=new_shell_D
        print(error)
        print(shell_D)


476
dp for Re_s 40887.85441053393
value for j 0.005227625651851436
2969.334062523191
N_ss is 2 While N_Tcc ia 14.306453541448098
value for N_b 20
1551.3953146841611
dp for G_w 1330.7333193979546
dp for cent baffle 0.37136448961590107
dp for baffle wind 0.3568709213828771
dp for entrance 0.18447760066177799
value for v_t 1.402819410155099
value for Re_t 75153.77031751792
value for Pr_t 3.901935483870968
value for f_turbulent 0.004783174614577113
value for Nu_tube 352.73168208201906
value for h_t_i 2212.005828295871
0.11887093066861441 5.0
476
dp for Re_s 40887.85441053393
value for j 0.005227625651851436
2969.334062523191
N_ss is 2 While N_Tcc ia 14.306453541448098
value for N_b 15
1551.3953146841611
dp for G_w 1330.7333193979546
dp for cent baffle 0.2736369923485587
dp for baffle wind 0.26765319103715784
dp for entrance 0.18447760066177799
value for v_t 1.402819410155099
value for Re_t 75153.77031751792
value for Pr_t 3.901935483870968
value for f_turbulent 0.004783174614577113
value f

In [28]:
mu_s = worksheet['H22'].value
mu_t = worksheet['K22'].value
mu_s_w = mu_s
D_sb = 3.1+0.004*shell_D
t_p = tpitch #p_ratio *Do
L_tb = 0.4 # Diametral Tube-Baffle Clearance
#t_p_effective 
if t_p_angle == 45:
    t_p_effective = t_p/(2**0.5)
else: t_p_effective = t_p

theta_Ds = 2*np.arccos(1-(2*b_cut)/100) # Baffle window angle
S_sb = (shell_D/1000)*(D_sb/1000)*(np.pi-0.5*theta_Ds)    # Shell to baffle leakage area
S_tb = (np.pi/4)*((((Do/1000)+(L_tb/1000))**2)-((Do/1000)**2))*tn*(1-F_w)  # Tube to baffle leakage area

r_L = (S_sb+S_tb)/S_m # ratio of leakage to cross flow
r_S = S_sb/(S_sb+S_tb) # ratio of shell-baffle to total area
#cross flow area 
S_m = (Lb_cut/1000)*((shell_D-D_otl)+(D_otl-Do)*(t_p-Do)/t_p_effective)/1000
Gs = (m_s/3600)/S_m
Re_s = (Do/1000)*Gs/(mu_s*0.001)
print('dp for Re_s '+str(Re_s))
# 3. correction factor for bundle bypass
# tube arrangement
if t_p_angle == 30:
    t_arrg = np.sqrt(3)/2
elif t_p_angle == 45:
    t_arrg = 1/np.sqrt(2)
else : t_arrg = 1
P_p = t_p * t_arrg # Tube row distance in flow direction
N_TCC = (shell_D/P_p)*(1-(2*b_cut)/100) #N_TCC number of tube rows between baffle
N_ss = int(N_TCC/6)
r_ss = N_ss/N_TCC # Nss Number of sealing strips
S_b = (Lb_cut/1000)*(shell_D-D_otl-(Do/2))/1000 #bundle pybass area
if Re_s < 100:
    C_j = 1.35
else: C_j = 1.25 # Correlation constant

if r_ss >= 0.5:
    j_b = 1
else: j_b = np.exp(-1*C_j*(S_b/S_m)*(1-((2*r_ss)**(1/3))))

# 4. Correction factor for adverse temperature gradient
N_tcw = (0.8/P_p)*(shell_D*b_cut/100-(shell_D-(D_otl-Do))/2)
N_b = 1 +int((L-(2*L_s*0.001)-(LB_in+LB_out)*0.001)/(Lb_cut*0.001)) # number of baffles
print('value for N_b '+str(N_b))
### Shell side pressure drop
b1 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{1}'],0).sum()
b2 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{2}'],0).sum()
b3 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{3}'],0).sum()
b4 = np.where((Re_s < j_const['Reynolds_max']) & (Re_s < j_const['Reynolds_max']) & (j_const['Layout'] == t_p_angle) ,j_const['b_{4}'],0).sum()
b = b3/(1+0.14*(Re_s**b4))
f_s = b1*((1.33/(t_p/Do))**b)*(Re_s**b2)
dp_shell_ideal = 2*f_s*(Gs**2)*N_TCC*(mu_s_w/mu_s)**0.14/(rho_s)/100000 #N_TCC number of tube rows between baffle
#print(dp_shell_ideal)

# 1. correction factor for baffle leakage
f_p = 0.8-0.15*(1+r_S) # r_s ratio of shell-baffle to total area
R_L = np.exp(-1.33*(1+r_S)*(r_L**f_p))# r_l ratio of  leakage area to cross flow

# pressure drop for an ideal window section
S_wg = (((shell_D/1000)**2)/8)*(theta_Ds-np.sin(theta_Ds)) #Gross window area
S_wt = tn*F_w*np.pi*((Do/1000)**2)/4 #Window area occupied with tubes
S_w = S_wg - S_wt # Net Cross flow area
G_w = (m_s/3600)/np.sqrt(S_m*S_w)
v_w = G_w/rho_s
print('dp for G_w '+str(G_w))
D_w = 4*S_w/(np.pi*(Do/1000)*tn*F_w+(shell_D/1000)*theta_Ds)

#pressure drop for turbluent flow in ideal window section
if Re_s >= 100:
    dp_window = N_b*R_L*(2+0.6*N_tcw)*(G_w**2)/(2*rho_s)/100000

    
else:
    dp_window = N_b*R_L*((26*G_w*Cp_s/rho_s)*(N_tcw/(tpitch-Do)/1000 + (Lb_cut/1000)/(D_w**2)) + (G_w**2)/rho_s)/100000
    

# 2. Correction factor for bundle bypass effect
if Re_s < 100:
    C_r = 4.5
else: C_r = 3.7

if r_ss >= 0.5:
    R_b = 1
else:
    R_b = np.exp(-1*C_r*(S_b/S_m)*(1-(2*r_ss)**(1/3)))

# 3. Correction for unequal baffle spacing inlet/ outlet
if Re_s < 100:
    n = 1 # Slope of friction factor curve
else: n = 0.2
R_s = (Lb_cut/LB_in)**(2-n) + (Lb_cut/LB_out)**(2-n) # correction factor

# Shell side pressure drop (Excluding nozzles)
dp_cent_baff = dp_shell_ideal*(N_b-1)*R_L*R_b # pressure drop in baffle windows
dp_baff_window = dp_window # pressure drop in baffle windows
dp_entrance_exit = dp_shell_ideal*R_s*R_b*(1+N_tcw/N_TCC) # pressure drop in entrance / exit baffles
print('dp for cent baffle '+str(dp_cent_baff))
print('dp for baffle wind '+str(dp_baff_window))
print('dp for entrance '+str(dp_entrance_exit))

total_dp_shell = dp_cent_baff + dp_baff_window + dp_entrance_exit
L*1000

dp for Re_s 78526.52317832608
value for N_b 0
dp for G_w 2553.699589262075
dp for cent baffle -0.0
dp for baffle wind 0.0
dp for entrance 0.0


89.81239851989811